Colab Pro A100で実行

In [ ]:
!pip install datasets transformers sentencepiece 

In [ ]:
import transformers
from datasets import Dataset
import pandas as pd
df = pd.read_csv("./train.csv")
df.head(1)

,prompt,target,category
0,\n質問は以下です。：戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?...,銅合金,incorrect2incorrect


In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

In [ ]:
train_df = df.copy()
train_df["prompt"] = train_df["prompt"].str.replace("[SEP]", " ", regex=False) # prompt部分に入っている[SEP]を消しておく

train_df["target"] = train_df["prompt"] + "[SEP]" + train_df["target"] # {prompt}[SEP]{target}の形で学習
train_df["len"] = train_df["target"].str.len()

In [ ]:
# max lengthが1024以下であることを確認
len(tokenizer.encode(train_df.sort_values(by="len", ascending=False).iloc[0]["target"]))

991

In [ ]:
n = len(train_df)
train_ds = Dataset.from_pandas(train_df.sample(n, random_state=102))
n

78389

In [ ]:

def tokenize_function(examples):
    encodings = tokenizer(examples["target"], padding="max_length", max_length=1024, truncation=True)

    labels = []
    for input_ids in encodings["input_ids"]:
        target_start_idx = input_ids.index(tokenizer.sep_token_id) + 1
        # prompt部分は学習する必要がないため、[SEP]までのlabelを[PAD]で埋める
        label =  [tokenizer.pad_token_id] * (target_start_idx) + input_ids[target_start_idx:]
        labels.append(label)
    encodings['labels'] = labels
    return encodings

In [ ]:
tokenized_train_ds = train_ds.map(tokenize_function,
                       batched=True,
                       num_proc=4,
                       remove_columns=df.columns.tolist()+['__index_level_0__']
                      )

#0:   0%|          | 0/20 [00:00<?, ?ba/s]

#1:   0%|          | 0/20 [00:00<?, ?ba/s]

#2:   0%|          | 0/20 [00:00<?, ?ba/s]

#3:   0%|          | 0/20 [00:00<?, ?ba/s]

In [ ]:

from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding
from typing import Union, List, Any, Dict, Optional, Mapping
import torch
class CustomCollatorForLanguageModeling(DataCollatorForLanguageModeling):
    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        # Handle dict or lists with proper padding and conversion to tensor.
        batch = self.tokenizer.pad(examples, return_tensors="pt", pad_to_multiple_of=self.pad_to_multiple_of)
        # If special token mask has been preprocessed, pop it from the dict.
        special_tokens_mask = batch.pop("special_tokens_mask", None)
        # labelsにはprompt部分をpadで埋めたlabelを使う。数値を-100にすることでloss計算されないようにする
        labels = torch.Tensor([e["labels"] for e in examples]).to(torch.long)
        if self.tokenizer.pad_token_id is not None:
            labels[labels == self.tokenizer.pad_token_id] = -100
        batch["labels"] = labels
        return batch

In [ ]:
data_collator = CustomCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )

batch_size = 1
total_steps = len(train_ds) // batch_size
total_steps

78389

In [ ]:
import torch
model_id = "rinna/japanese-gpt-1b"
model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:

model_name = f"qa-refine-japanese-gpt-1b"
training_args = TrainingArguments(
    model_name,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    num_train_epochs=1,
    warmup_steps=total_steps // 20,
    save_total_limit=5,
    no_cuda=False,
    fp16=True,
    gradient_accumulation_steps=8,
    save_steps=10000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train_ds,
)

trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: len. If len are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 78389
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 8
  Total optimization steps = 9798
  Number of trainable parameters = 1302712320


Step,Training Loss
500,2.691900
1000,0.674500
1500,0.449500
2000,0.396400
2500,0.412900
3000,0.388700
3500,0.431700
4000,0.434300
4500,0.434300
5000,0.391700




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9798, training_loss=0.4863641485434986, metrics={'train_runtime': 12592.346, 'train_samples_per_second': 6.225, 'train_steps_per_second': 0.778, 'total_flos': 5.820525040411607e+17, 'train_loss': 0.4863641485434986, 'epoch': 1.0})

In [ ]:
prompt = train_df.iloc[1]["prompt"] + "[SEP]"
input_ids = tokenizer.encode(prompt, return_tensors='pt', add_special_tokens=False).cuda()
out = model.generate(
    input_ids,
    max_length=1024,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
  )
display(
    tokenizer.decode(out[0], skip_special_tokens=False)
    )

'質問は以下です。:手塚治虫の出身地はどこになりますか? すでに答えの候補があります。:兵庫県宝塚市 必要な場合のみ、以下の文脈情報を使ってこの答えを改良することができます。 --- 0.999[UNK]と1の等価性は、実数の体系(これは解析学ではもっとも一般的に用いられる体系である)に0でない無限小が存在しないことと深く関係している。 一方、超実数の体系のように0でない無限小を含む別の数体系もある。 そのような体系の大半は、標準的な解釈のもとで式0.999[UNK]の値は1に等しくなるが、一部の体系においては記号"0.999[UNK]"に別の解釈を与えて1よりも無限小だけ小さいようにすることができる。 等式0.999[UNK]=1は数学者に長く受け入れられ、一般の数学教育の一部であったにも拘らず、これを十分直観に反するものと見做して、疑念や拒絶反応を示す学徒もいる。 このような懐疑論は、「この等式を彼らに納得させることがいかに難しいか」が数学教育の様々な研究の主題となることに正当性を与える程度に当たり前に存在している。 --- この文脈情報により、元の答えを改良して質問に答えてください。 文脈情報が有用でない場合は元の答えをそのまま返してください。[SEP] 兵庫県宝塚市</s>'

In [ ]:
model.to(torch.float16)

In [ ]:
model.dtype

torch.float16

In [ ]:
model.save_pretrained("./qa-refine-japanese-gpt-1b")